In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Format mẫu
- Câu 1: Ứng với một thời gian cho trước, hãy rút trích đặc trưng cho mẫu….
Tổ chức thành chương trình con 
- Câu 2: Viết chương trình chuẩn bị bộ dữ liệu M hàng và N cột, trong đó Các cột là các đặc trưng như sau:
- Câu 3: Chia dữ liệu thành tập train/val/ test theo quy tắc …..
- Câu 4: Các đặc trưng sẽ được xử lý theo quy tắc …. . Hãy xây dựng chương trình định nghĩa đối tượng CoumnTransfor để thực hiện viẹc tiền xử lý dữ liệu
- Câu 5: Thực hiện phân loại với Pipeline định nghĩa như sau. Hãy xác định những tham số điều khiển nào cần tinh chỉnh
- Câu 6: Huấn luyện và lựa chọn mô hình tốt nhất bằng Grid search từ dữ liệu train / val
- Câu 7: Đánh giá mô hình dựa vào dữ liệu test
## Đề mẫu
- Câu 1: Cho thời điểm t (tính theo đơn vị giờ), hãy truy vấn các dữ liệu trong khoảng thời gian từ t - T đến t, trong đó T là giá trị đã biết, tính bằng giờ.
- Câu 2: Giả sử M là số thời điểm (t) có thể lấy được trong bộ dữ liệu và N là số dịch vụ, hãy tạo một ma trận có kích thước M * (4 * N) chứa các đặc trưng đã rút trích ở câu 5 cho mỗi dịch vụ: 
    5. Rút Trích Các Đặc Trưng Từ Mỗi Nhóm
    Với mỗi nhóm dịch vụ, rút trích các đặc trưng sau:
    
    a. Tổng số lần gọi (Total Calls)
    
    b. Tổng số lần gọi thành công (Successful Calls)
    
    c. Trị trung bình thời gian thực thi (Average Execution Time)
    
    d. Trị trung bình dữ liệu sử dụng (Average Data Usage)
    
    datetime chia hour và day

- Câu 3: Chia dữ liệu thành tập train/val/ test theo quy tắc (nếu dự đoán phải dùng time series split, ngược lại train test split). Từ việc xây dựng chương trình định nghĩa đối tượng CoumnTransfor để thực hiện viẹc tiền xử lý dữ liệu với việc embeding hour theo (x,y, hình tròn để tìm khoảng cách) còn day sẽ dùng one hot encoder 
- Câu 5: Thực hiện phân loại với Pipeline định nghĩa như sau. Hãy xác định những tham số điều khiển nào cần tinh chỉnh (học giám sát)
- Câu 6: Huấn luyện và lựa chọn mô hình tốt nhất bằng Grid search từ dữ liệu train / val
- Câu 7: Đánh giá mô hình dựa vào dữ liệu test


In [ ]:
df = pd.read_csv("clean_feature.csv")
df = df.dropna()
df = df.drop_duplicates()

# df["datetime"] = pd.to_datetime(df["Day"]) + pd.to_timedelta(df["hour"], unit="h")
